In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
import torch

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 번역 함수 정의 (배치 처리)
def batch_translate(texts, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    translated = model.generate(**inputs)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# 데이터 불러오기
data = pd.read_csv("/aiffel/aiffel/dktc/data/train.csv")

# 중복 데이터 확인 및 삭제
data = data.drop_duplicates(subset=['conversation'])

# 한국어 -> 프랑스어 -> 영어 -> 한국어 번역 증강 (배치 처리)
def augment_korean_to_french_to_english_to_korean(texts):
    ko_to_fr_model_name = "Helsinki-NLP/opus-mt-ko-fr"
    fr_to_en_model_name = "Helsinki-NLP/opus-mt-tc-big-fr-en"
    en_to_ko_model_name = "Helsinki-NLP/opus-mt-tc-big-en-ko"
    
    translated_to_french = batch_translate(texts, ko_to_fr_model_name)
    translated_to_english = batch_translate(translated_to_french, fr_to_en_model_name)
    return batch_translate(translated_to_english, en_to_ko_model_name)

# 데이터 증강 리스트 정의
augmented_texts_french = []

# 데이터 증강 진행 (배치 처리)
batch_size = 20
for i in tqdm(range(0, len(data), batch_size), desc="Augmenting data"):
    batch_texts = data['conversation'].iloc[i:i+batch_size].tolist()
    augmented_texts_french.extend(augment_korean_to_french_to_english_to_korean(batch_texts))

# 증강된 데이터를 원본 데이터와 병합
augmented_df_french = pd.DataFrame({
    'class': data['class'],
    'conversation': augmented_texts_french
})
data_augmented = pd.concat([data, augmented_df_french])

# 중복 데이터 제거
data_augmented = data_augmented.drop_duplicates(subset=['conversation'])

# 결과 저장
data_augmented.to_csv("train_augmented_fra.csv", index=False)


Augmenting data:  46%|████▌     | 89/193 [1:27:43<1:50:42, 63.87s/it]

In [1]:
import torch
torch.cuda.is_available()

True